In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_S.cerevisiae_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(2883, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []

# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")

    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,

    )
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

#  Show TP, TN, FP, FN
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_S.cerevisiae_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_S.cerevisiae_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


163/163 ━━━━━━━━━━━━━━━━━━━━ 36s 71ms/step - AUC: 0.6721 - categorical_accuracy: 0.6331 - distill_loss1: 0.0329 - distill_loss2: 0.0798 - loss: 0.5000 - student_loss: 0.6488 - val_AUC: 0.7014 - val_categorical_accuracy: 0.6713 - val_loss: 0.5000 - val_student_loss: 0.6141
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - AUC: 0.7092 - categorical_accuracy: 0.6696 - distill_loss1: 0.0308 - distill_loss2: 0.0745 - loss: 0.5000 - student_loss: 0.6157 - val_AUC: 0.7429 - val_categorical_accuracy: 0.6713 - val_loss: 0.5000 - val_student_loss: 0.5356
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - AUC: 0.7368 - categorical_accuracy: 0.6699 - distill_loss1: 0.0308 - distill_loss2: 0.0716 - loss: 0.5000 - student_loss: 0.5938 - val_AUC: 0.7583 - val_categorical_accuracy: 0.6817 - val_loss: 0.5000 - val_student_loss: 0.3722
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - AUC: 0.7855 - categorical_accuracy: 0.7090 - distill_loss1: 0.0291 - distill_loss2: 0.0678 - loss: 0.5000

...

163/163 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - AUC: 0.4687 - categorical_accuracy: 0.4780 - distill_loss1: 0.0362 - distill_loss2: 0.0866 - loss: 0.5000 - student_loss: 0.6808 - val_AUC: 0.6432 - val_categorical_accuracy: 0.6493 - val_loss: 0.5000 - val_student_loss: 0.6065
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - AUC: 0.7418 - categorical_accuracy: 0.6837 - distill_loss1: 0.0301 - distill_loss2: 0.0737 - loss: 0.5000 - student_loss: 0.6105 - val_AUC: 0.7096 - val_categorical_accuracy: 0.6493 - val_loss: 0.5000 - val_student_loss: 0.5543
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - AUC: 0.7868 - categorical_accuracy: 0.7126 - distill_loss1: 0.0274 - distill_loss2: 0.0682 - loss: 0.5000 - student_loss: 0.5740 - val_AUC: 0.7363 - val_categorical_accuracy: 0.6528 - val_loss: 0.5000 - val_student_loss: 0.4462
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - AUC: 0.8001 - categorical_accuracy: 0.7164 - distill_loss1: 0.0264 - distill_loss2: 0.0651 - loss: 0.5000 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


163/163 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - AUC: 0.6280 - categorical_accuracy: 0.6075 - distill_loss1: 0.0326 - distill_loss2: 0.0812 - loss: 0.5000 - student_loss: 0.6561 - val_AUC: 0.7903 - val_categorical_accuracy: 0.7188 - val_loss: 0.5000 - val_student_loss: 0.6186
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - AUC: 0.7333 - categorical_accuracy: 0.6840 - distill_loss1: 0.0309 - distill_loss2: 0.0747 - loss: 0.5000 - student_loss: 0.6206 - val_AUC: 0.8244 - val_categorical_accuracy: 0.7188 - val_loss: 0.5000 - val_student_loss: 0.5516
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - AUC: 0.7616 - categorical_accuracy: 0.6834 - distill_loss1: 0.0293 - distill_loss2: 0.0706 - loss: 0.5000 - student_loss: 0.5965 - val_AUC: 0.8432 - val_categorical_accuracy: 0.7431 - val_loss: 0.5000 - val_student_loss: 0.4656
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - AUC: 0.7812 - categorical_accuracy: 0.7024 - distill_loss1: 0.0265 - distill_loss2: 0.0660 - loss: 0.500

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    # Re-initialize the distiller for each fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

    # Save result of each fold
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_S.cerevisiae_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_S.cerevisiae_KD2_Succ1.xlsx'")

Training on fold 1/10...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


163/163 [==============================] - 3s 6ms/step - loss: 0.7077 - categorical_accuracy: 0.6130 - auc: 0.6477
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6901 - categorical_accuracy: 0.6207 - auc: 0.6548
Epoch 3/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6719 - categorical_accuracy: 0.6307 - auc: 0.6686
Epoch 4/50
163/163 [==============================] - 1s 6ms/step - loss: 0.6676 - categorical_accuracy: 0.6426 - auc: 0.6661
Epoch 5/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6555 - categorical_accuracy: 0.6399 - auc: 0.6737
Epoch 6/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6537 - categorical_accuracy: 0.6349 - auc: 0.6773
Epoch 7/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6461 - categorical_accuracy: 0.6446 - auc: 0.6844
Epoch 8/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6458 - categorical_accuracy: 0.6542 - auc: 0.6843
Epo

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


163/163 [==============================] - 2s 5ms/step - loss: 0.8147 - categorical_accuracy: 0.5563 - auc: 0.5800
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7198 - categorical_accuracy: 0.6110 - auc: 0.6490
Epoch 3/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7232 - categorical_accuracy: 0.6052 - auc: 0.6357
Epoch 4/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6846 - categorical_accuracy: 0.6157 - auc: 0.6559
Epoch 5/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6647 - categorical_accuracy: 0.6376 - auc: 0.6787
Epoch 6/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6636 - categorical_accuracy: 0.6419 - auc: 0.6764
Epoch 7/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6539 - categorical_accuracy: 0.6515 - auc: 0.6854
Epoch 8/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6587 - categorical_accuracy: 0.6442 - auc: 0.6793
Epo

...

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


163/163 [==============================] - 2s 5ms/step - loss: 0.7657 - categorical_accuracy: 0.5938 - auc: 0.6196
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7039 - categorical_accuracy: 0.6189 - auc: 0.6541
Epoch 3/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6926 - categorical_accuracy: 0.6193 - auc: 0.6574
Epoch 4/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6900 - categorical_accuracy: 0.6254 - auc: 0.6529
Epoch 5/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6708 - categorical_accuracy: 0.6308 - auc: 0.6670
Epoch 6/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6691 - categorical_accuracy: 0.6312 - auc: 0.6657
Epoch 7/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6596 - categorical_accuracy: 0.6358 - auc: 0.6746
Epoch 8/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6641 - categorical_accuracy: 0.6343 - auc: 0.6693
Epo

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


163/163 [==============================] - 2s 5ms/step - loss: 0.8294 - categorical_accuracy: 0.5723 - auc: 0.5995
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7833 - categorical_accuracy: 0.5699 - auc: 0.6070
Epoch 3/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7492 - categorical_accuracy: 0.5985 - auc: 0.6229
Epoch 4/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7144 - categorical_accuracy: 0.6066 - auc: 0.6409
Epoch 5/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6967 - categorical_accuracy: 0.6116 - auc: 0.6502
Epoch 6/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6934 - categorical_accuracy: 0.6127 - auc: 0.6491
Epoch 7/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6661 - categorical_accuracy: 0.6412 - auc: 0.6754
Epoch 8/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6663 - categorical_accuracy: 0.6324 - auc: 0.6681
Epo

ModuleNotFoundError: No module named 'openpyxl'